In [ ]:
import itertools as it
import operator
import re
import subprocess
import tempfile
import time
from collections import Counter, defaultdict
from datetime import timedelta
from functools import partial
from pathlib import Path

import awkward as ak
import duckdb
import gfapy
import holoviews as hv
import ibis
import matplotlib.pyplot as plt
import numba
import numpy as np
import pandas as pd
import parasail
import pod5
import polars as pl
import pyabpoa
import pyarrow as pa
import pyarrow.compute as pc
import pyarrow.dataset as ds
import pyfastx
import pysam
import spoa
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from pyarrow import csv
from pywfa import WavefrontAligner
from tqdm.auto import tqdm, trange

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.sequencing.align as align
import paulssonlab.sequencing.cigar as scigar
import paulssonlab.sequencing.consensus as con
import paulssonlab.sequencing.gfa as sgfa
import paulssonlab.sequencing.io as sio
import paulssonlab.sequencing.processing as processing
import paulssonlab.sequencing.uuid as uuid
from paulssonlab.util.sequence import reverse_complement

In [ ]:
hv.extension("bokeh")

In [ ]:
%load_ext pyinstrument
import line_profiler
import pyinstrument

%load_ext line_profiler

In [ ]:
pl.enable_string_cache()

# Config

In [ ]:
gfa_filename = "/home/jqs1/scratch3/jqs1/sequencing/230930_alignment_test/barcode.gfa"

In [ ]:
gfa = gfapy.Gfa.from_file(gfa_filename)

# Duplex pairing

In [ ]:
# bam_filename = "/home/jqs1/scratch3/jqs1/sequencing/230930_alignment_test/230707_repressilators/channel-135_merged.bam"
# bam_filename = "/home/jqs1/scratch3/jqs1/sequencing/230922_bcd_rbses_constitutive/20230922_1104_1A_PAQ83451_8d610a8c/bam_pass/channel-100_merged.bam"
bam_filename = "/home/jqs1/scratch/sequencing/230707_repressilators/20230707_2040_MN35044_FAS94231_25542e0d/_temp/channel-1_merged.bam"
gaf_filename = "/home/jqs1/scratch/sequencing/230707_repressilators/20230707_2040_MN35044_FAS94231_25542e0d/_temp/channel-1_merged.gaf"

In [ ]:
# %%time
# bam = pysam.AlignmentFile(bam_filename, check_sq=False)
# bam.reset()
# reads = [{"name": read.query_name, **dict(read.tags)} for read in tqdm(bam.fetch(until_eof=True))]

In [ ]:
# %%time
# reads = pa.Table.from_batches(tqdm(sio.iter_bam_and_gaf(bam_filename, gaf_filename, include_unaligned=False)))

In [ ]:
%%time
df = processing.load_pairing_data(bam_filename, gaf_filename)

In [ ]:
import networkx as nx

include = []
include_prefix = []
exclude = ["UNS9", "BC:UPSTREAM", "BC:JUNCTION", "BC:T7_TERM", "BC:SPACER2"]
exclude_prefix = []
###
gfa = gfapy.Gfa.from_file(gfa_filename)
gfa = sgfa.filter_gfa(gfa, include, include_prefix, exclude, exclude_prefix)
graph = sgfa.gfa_to_dag(gfa)
# weakly_connected_components is a generator, so only compute once
wccs = list(nx.weakly_connected_components(graph))
forward_segments = sgfa.dag_forward_segments(graph, wccs=wccs)
endpoints = sgfa.dag_endpoints(graph, wccs=wccs)

In [ ]:
(
    path_filter,
    _,
    reverse_path_mapping,
) = processing._segments_for_normalize_path(forward_segments)

In [ ]:
df2 = df.filter(pl.col("qs") >= 8)

In [ ]:
df_pairs = processing.find_duplex_pairs(
    df2, timedelta(seconds=3), forward_segments, endpoints=endpoints
)

In [ ]:
df_pairs

In [ ]:
dupes = (
    pl.concat([df_pairs["name"], df_pairs["name_right"]])
    .value_counts()
    .filter(pl.col("counts") > 1)
)
dupes

In [ ]:
df_pairs.filter(pl.col("name").is_in(dupes["name"]))

In [ ]:
df_pairs.filter(pl.col("name_right").is_in(dupes["name"]))

In [ ]:
dts = [*np.linspace(0, 1, 11), *np.arange(2, 20)]
bad = [
    len(
        processing.find_duplex_pairs(
            df2, timedelta(seconds=float(dt)), forward_segments, endpoints=endpoints
        )
    )
    for dt in tqdm(dts)
]

In [ ]:
plt.plot(dts, good, marker="x")
plt.plot(dts, bad, marker="x");

In [ ]:
plt.plot(dts, np.array(good) / np.array(bad), marker="x");

In [ ]:
good[12] / good[10]

In [ ]:
dts[10]